import numpy as np
import scipy as sp
from statsmodels.stats.power import NormalIndPower




In [1]:
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection as fdr
import numpy as np
import scipy.stats as stats
methods = ["FiLM", "TAG", "GCN", "N2V+MLP", "MLP"]
diseases = ["Cardiovascular Disease", "Immune Dysregulation", "Body Mass Disorders", "Insulin Disorders", "Diabetes"]
outer_dfs = []
global_pvals = []
for value in diseases:
    results_df = pd.DataFrame(columns=["Mendelian/Method", "CS", "log odds", "a", "b", "c", "d", "se", "delta_log_odds (C-M)", "se_delta", "Z", "pval unadjusted", "pval adjusted (FDR)", "result"])
    df= pd.read_csv("../statistical_dump/{}_ors_mko.tsv".format(value), sep="\t", header=0)
    method_list = ["Mendelian"]
    cs_list = ["-"]
    log_odds = []
    a_list = []
    b_list = []
    c_list = []
    d_list = []
    se_list = []
    delta_log_odds_list = [np.nan]
    se_delta_list = [np.nan]
    zval_list = [np.nan]
    pval_list = [np.nan]
    result_list = ["-"]

    need_set_mendelians = True
    for method in methods:
        method_list.append(method)
        cs = 12
        significant = False

        a = "N Candidate/Mendelian and MKO"
        b = "N Candidate/Mendelian not MKO"
        c = "N Not Candidate/Mendelian and MKO"
        d = "N Not Candidate/Mendelian Not MKO"
        pval = "pval adjusted (FDR)"

        while not significant:
            cs -= 1
            candidate_row = df[(df["Method"] == method) & (df["CS/Mendelian"] == "CS >= {}".format(cs))]
            significant = candidate_row[pval].item() < 0.05

        cs_list.append(cs)

        mendelian_row = df.iloc[11]

        log_odds_candidates = np.log(candidate_row["OR"].item())
        log_odds_mendelians = np.log(mendelian_row["OR"].item())
        delta = log_odds_candidates - log_odds_mendelians

        var_candidates = np.sum([1 / candidate_row[column] for column in [a, b, c, d]])
        var_mendelian = np.sum([1 / mendelian_row[column] for column in [a, b, c, d]])

        if need_set_mendelians:
            log_odds.append(log_odds_mendelians)
            a_list.append(mendelian_row[a].item())
            b_list.append(mendelian_row[b].item())
            c_list.append(mendelian_row[c].item())
            d_list.append(mendelian_row[d].item())
            se_list.append(np.sqrt(var_mendelian))
            need_set_mendelians = False

        log_odds.append(log_odds_candidates)
        a_list.append(candidate_row[a].item())
        b_list.append(candidate_row[b].item())
        c_list.append(candidate_row[c].item())
        d_list.append(candidate_row[d].item())
        se_list.append(np.sqrt(var_candidates))

        n1 = np.sum([candidate_row[column] for column in [a, b, c, d]])
        n2 = np.sum([mendelian_row[column] for column in [a, b, c, d]])

        se_delta = np.sqrt(var_candidates + var_mendelian)
        zval = delta / se_delta
        pval = stats.norm.sf(np.abs(zval)) * 2

        delta_log_odds_list.append(delta)
        se_delta_list.append(se_delta)
        zval_list.append(zval)
        pval_list.append(pval)

    results_df["Mendelian/Method"] =  method_list
    results_df["CS"] = cs_list
    results_df["log odds"] = log_odds
    results_df["a"] = a_list
    results_df["b"] = b_list
    results_df["c"] = c_list
    results_df["d"] = d_list
    results_df["se"] = se_list
    results_df["delta_log_odds (C-M)"] = delta_log_odds_list
    results_df["se_delta"] = se_delta_list
    results_df["Z"] = zval_list
    results_df["pval unadjusted"] = pval_list
    #print("{}: CS >= {}: {}".format(method, cs, result))
    #print("Power: {}".format(pwr.power(effect_size=zval, nobs1=n1, alpha=0.05, ratio=n2/n1)))
    #print("Pval: {}".format(pval))
    outer_dfs.append(results_df)
    global_pvals.extend(np.asarray(pval_list)[~np.isnan(pval_list)])

/home/icb/florin.ratajczak/anaconda3/envs/compat/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in long_scalars


In [4]:
results_df

,Mendelian/Method,CS,log odds,a,b,c,d,se,delta_log_odds (C-M),se_delta,Z,pval unadjusted,pval adjusted (FDR),result
0,GWAS,-,NaN,0,0,719,13822,inf,NaN,NaN,NaN,NaN,NaN,NaN
1,FiLM,8,1.104594,5,32,714,13790,0.482414,NaN,inf,NaN,NaN,NaN,NaN
2,TAG,6,0.910273,10,78,709,13744,0.338088,NaN,inf,NaN,NaN,NaN,NaN
3,GCN,11,2.452986,6,10,713,13812,0.517824,NaN,inf,NaN,NaN,NaN,NaN
4,N2V+MLP,11,1.680828,5,18,714,13804,0.506980,NaN,inf,NaN,NaN,NaN,NaN
5,MLP,10,0.777488,12,107,707,13715,0.306866,NaN,inf,NaN,NaN,NaN,NaN


In [19]:
print(np.sum(np.asarray(global_pvals) < 0.05))

pvals_adj = fdr(global_pvals)[1]

(np.sum(np.asarray(pvals_adj) < 0.05))


15


13

In [20]:
for df, df_pvals_adj, disease in zip(outer_dfs, np.array_split(pvals_adj, len(outer_dfs)), diseases):
    print(len(df))
    print(df_pvals_adj)
    df.loc[~np.isnan(df["pval unadjusted"]), "pval adjusted (FDR)"] = df_pvals_adj
    result = []
    zvals = df.loc[~np.isnan(df["pval unadjusted"]), "Z"]
    for pval, zval in zip(df_pvals_adj, zvals):
        if pval < 0.05:
            if zval < 0:
                result.append("less")
            else:
                result.append("greater")
        else:
            result.append("equal")
    df.loc[~np.isnan(df["pval unadjusted"]), "result"] = result

    df.to_csv("../statistical_dump/{}_difference_gwas.tsv".format(disease), sep="\t", index=False)

    

6
[0.05901276 0.00050454 0.09131208 0.01669394 0.18649947]
6
[6.55189216e-06 7.03673346e-07 2.01962254e-02 5.23222836e-07
 2.01962254e-02]
6
[0.12479901 0.02019623 0.0394252  0.0011041  0.05901276]
6
[5.04542114e-04 8.30646579e-01 9.13120847e-02 2.05650803e-01
 5.82914398e-01]
6
[0.13350469 0.12479901 0.00032639 0.02019623 0.17821412]


In [7]:
df.head()

,Mendelian/Method,CS,log odds,a,b,c,d,se,delta_log_odds (C-M),se_delta,Z,pval unadjusted,pval adjusted (FDR),result
0,Mendelian,-,2.560246,70,104,719,13822,0.159262,NaN,NaN,NaN,NaN,NaN,NaN
1,FiLM,8,1.104594,5,32,714,13790,0.482414,-1.455652,0.508023,-2.865326,0.004166,0.013018,less
2,TAG,6,0.910273,10,78,709,13744,0.338088,-1.649973,0.373722,-4.414978,0.000010,0.000063,less
3,GCN,11,2.452986,6,10,713,13812,0.517824,-0.107260,0.541762,-0.197984,0.843058,0.874897,equal
4,N2V+MLP,11,1.680828,5,18,714,13804,0.506980,-0.879418,0.531406,-1.654888,0.097947,0.14404,equal


In [17]:
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection as fdr
import numpy as np
import scipy.stats as stats
methods = ["FiLM", "TAG", "GCN", "N2V+MLP", "MLP"]
diseases = ["Cardiovascular Disease", "Immune Dysregulation", "Body Mass Disorders", "Insulin Disorders", "Diabetes"]
outer_dfs = []
global_pvals = []
for value in diseases:
    results_df = pd.DataFrame(columns=["GWAS/Method", "CS", "log odds", "a", "b", "c", "d", "se", "delta_log_odds (C-G)", "se_delta", "Z", "pval unadjusted", "pval adjusted (FDR)", "result"])
    df= pd.read_csv("../statistical_dump/{}_ors_mko_gwas.tsv".format(value), sep="\t", header=0)
    method_list = ["GWAS"]
    cs_list = ["-"]
    log_odds = []
    a_list = []
    b_list = []
    c_list = []
    d_list = []
    se_list = []
    delta_log_odds_list = [np.nan]
    se_delta_list = [np.nan]
    zval_list = [np.nan]
    pval_list = [np.nan]
    result_list = ["-"]

    need_set_mendelians = True
    for method in methods:
        method_list.append(method)
        cs = 12
        significant = False

        a = "N Candidate/Mendelian and MKO"
        b = "N Candidate/Mendelian not MKO"
        c = "N Not Candidate/Mendelian and MKO"
        d = "N Not Candidate/Mendelian Not MKO"
        pval = "pval adjusted (FDR)"

        while not significant:
            cs -= 1
            candidate_row = df[(df["Method"] == method) & (df["CS/Mendelian"] == "CS >= {}".format(cs))]
            significant = candidate_row[pval].item() < 0.05

        cs_list.append(cs)

        mendelian_row = df.iloc[0]

        log_odds_candidates = np.log(candidate_row["OR"].item())
        log_odds_mendelians = np.log(mendelian_row["OR"].item())
        delta = log_odds_candidates - log_odds_mendelians

        var_candidates = np.sum([1 / candidate_row[column] for column in [a, b, c, d]])
        var_mendelian = np.sum([1 / mendelian_row[column] for column in [a, b, c, d]])

        if need_set_mendelians:
            log_odds.append(log_odds_mendelians)
            a_list.append(mendelian_row[a].item())
            b_list.append(mendelian_row[b].item())
            c_list.append(mendelian_row[c].item())
            d_list.append(mendelian_row[d].item())
            se_list.append(np.sqrt(var_mendelian))
            need_set_mendelians = False

        log_odds.append(log_odds_candidates)
        a_list.append(candidate_row[a].item())
        b_list.append(candidate_row[b].item())
        c_list.append(candidate_row[c].item())
        d_list.append(candidate_row[d].item())
        se_list.append(np.sqrt(var_candidates))

        n1 = np.sum([candidate_row[column] for column in [a, b, c, d]])
        n2 = np.sum([mendelian_row[column] for column in [a, b, c, d]])

        se_delta = np.sqrt(var_candidates + var_mendelian)
        zval = delta / se_delta
        pval = stats.norm.sf(np.abs(zval)) * 2

        delta_log_odds_list.append(delta)
        se_delta_list.append(se_delta)
        zval_list.append(zval)
        pval_list.append(pval)

    results_df["GWAS/Method"] =  method_list
    results_df["CS"] = cs_list
    results_df["log odds"] = log_odds
    results_df["a"] = a_list
    results_df["b"] = b_list
    results_df["c"] = c_list
    results_df["d"] = d_list
    results_df["se"] = se_list
    results_df["delta_log_odds (C-G)"] = delta_log_odds_list
    results_df["se_delta"] = se_delta_list
    results_df["Z"] = zval_list
    results_df["pval unadjusted"] = pval_list
    #print("{}: CS >= {}: {}".format(method, cs, result))
    #print("Power: {}".format(pwr.power(effect_size=zval, nobs1=n1, alpha=0.05, ratio=n2/n1)))
    #print("Pval: {}".format(pval))
    outer_dfs.append(results_df)
    global_pvals.extend(np.asarray(pval_list)[~np.isnan(pval_list)])